In [6]:
import pandas as pd
from datetime import datetime
import numpy as np
import glob
import os
import xlrd
import re
import openpyxl
import random

# custom file that maps state names to abbreviations
from abbreviation_conversion import abbrev_to_us_state

## Clean up prescription data


In [7]:
def find_year(TRANSACTION_DATE):
    """
    Args:
        TRANSACTION_DATE (str): date in format MMDDYYYY

    Returns:
        int: year
    """
    TRANSACTION_DATE = str(TRANSACTION_DATE)
    
    return int(TRANSACTION_DATE[-4:])

# quick test 
print(f"testing the find_year function: {find_year(12202019)}")


def find_month(TRANSACTION_DATE):
    """
    Args:
        TRANSACTION_DATE (str): date in format MMDDYYYY

    Returns:
        int: month
    """
    TRANSACTION_DATE = str(TRANSACTION_DATE)

    if len(TRANSACTION_DATE) == 8:
        return int(TRANSACTION_DATE[:2])
    else:
        return int(TRANSACTION_DATE[:1])
    

# quick test 
print(f"testing the find_month function: {find_month(12202019)}")

testing the find_year function: 2019
testing the find_month function: 12


In [8]:
# to load in the data, we need to truncate the amount of columns we use as well as the states
cols_to_keep = ["REPORTER_DEA_NO", "BUYER_STATE", "BUYER_ZIP", "BUYER_COUNTY", "DRUG_CODE", "TRANSACTION_CODE", "DRUG_NAME", "QUANTITY", "TRANSACTION_DATE", "Product_Name"]

# we know we need Florida, Texas, and Washington
states = ["FL", "TX", "WA"]
# since we are normalizing based on population, I think we should pick states that are regionally close to our target states
# we can change this later as a group, but I have these selected below:

# Florida comparison states
fl_states = ["PA", "MI", "NC"]

# Texas comparison states
tx_states = ["IL", "MA", "MI"]

# Washington comparison states
wa_states = ["NC", "CO", "MD"]

# create list of all states to use
variable_states = []
variable_states.extend(fl_states)
variable_states.extend(tx_states)
variable_states.extend(wa_states)

# append variable states to our original list
states.extend(variable_states)


# create separate list of only florida and washington states for prescription data
prescription_states = [state for state in states if state not in ["IL", "MA", "MI", "TX"]]

# NC is appearing twice as it's a comparison state for both target states
# making this a set will remove the duplicate
prescription_states = list(set(prescription_states))

In [9]:
# now, load in our data as an iterator so we can load in chunks
it = pd.read_csv("00_source_data/arcos_all_washpost.tsv", chunksize=500_000, sep='\t', usecols = cols_to_keep) # may have to change chunksize depending on your computer's memory

# init empty dataframe
df = pd.DataFrame()

for chunk in it:
    # extract year out of date column
    chunk["year"] = chunk["TRANSACTION_DATE"].apply(lambda x: find_year(x))
    chunk["month"] = chunk["TRANSACTION_DATE"].apply(lambda x: find_month(x))

    # ensure we're working in the correct date range
    filtered_chunk = chunk[chunk["year"] > 2002]
    filtered_chunk = filtered_chunk[filtered_chunk["year"] < 2016]

    # filter out the states we want
    filtered_chunk = filtered_chunk[filtered_chunk["BUYER_STATE"].isin(prescription_states)]

    df = df.append(filtered_chunk)
    break
df


C:\Users\abzdel\AppData\Local\Temp/ipykernel_52032/218960981.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(filtered_chunk)


,REPORTER_DEA_NO,BUYER_STATE,BUYER_ZIP,BUYER_COUNTY,TRANSACTION_CODE,DRUG_CODE,DRUG_NAME,QUANTITY,TRANSACTION_DATE,Product_Name,year,month
18,PA0021179,CO,81005,PUEBLO,S,9193,HYDROCODONE,10.0,1122009,HYDROCODONE BITARTRATE & ACETA 5MG/,2009,1
4745,PB0034861,FL,33460,PALM BEACH,S,9143,OXYCODONE,1.0,8182006,OXYCODONE HCL 40MG TABS,2006,8
4746,PB0034861,FL,33460,PALM BEACH,S,9143,OXYCODONE,8.0,11292006,ENDOCET - 10MG OXYCODONE.HCL/325MG A,2006,11
4747,PB0034861,FL,33460,PALM BEACH,S,9143,OXYCODONE,3.0,2062007,OXYCODONE HCL 80MG TABS,2007,2
4748,PB0034861,FL,33460,PALM BEACH,S,9143,OXYCODONE,3.0,3012007,OXYCODONE HCL 80MG TABS,2007,3
...,...,...,...,...,...,...,...,...,...,...,...,...
494070,RC0231148,FL,32832,ORANGE,S,9193,HYDROCODONE,7.0,7122007,"HYDROCODONE BIT./ACET.,10MG & 325MG/",2007,7
494206,RC0231148,WA,99328,COLUMBIA,S,9193,HYDROCODONE,80.0,10032006,HYDROCODONE BITARTRATE/APAP 10MG/650,2006,10
494207,RC0231148,WA,99328,COLUMBIA,S,9193,HYDROCODONE,20.0,12132006,HYDROCODONE BITARTRATE/APAP 5MG/500M,2006,12
494208,RC0231148,WA,99328,COLUMBIA,S,9193,HYDROCODONE,82.0,3112008,HYDROCODONE BITARTRATE/APAP 10MG/650,2008,3


### Loading in individual state prescription data

The Washington Post article associated with our data states that data from 2013 and 2014 was only recently added. Resultingly, we found that it was missing from the large dataset of all states. However, upon further digging, we found that these years were present **on an individual state level**, so we will load these in and concatenate them with our larger dataframe above.

Without chunking, the below takes 30 seconds for each file to load in. With chunking, this is reduced to about 4 seconds per record, so please make sure to leave this in its current format.

In [10]:
path = r'00_source_data/state_prescriptions' # point to correct folder
filenames = glob.glob(path + "/*.csv") # select all text files in folder

assert len(filenames) == 7, "There should be 7 files in the folder"

missing_years = pd.DataFrame() # empty df - will store data from all txt files

for f in filenames:

    it = pd.read_csv(f, chunksize=500_000, usecols = cols_to_keep) # may have to change chunksize depending on your computer's memory
    temp_df = pd.DataFrame()

    for chunk in it:
        # extract year out of date column
        chunk["year"] = chunk["TRANSACTION_DATE"].apply(lambda x: find_year(x))
        chunk["month"] = chunk["TRANSACTION_DATE"].apply(lambda x: find_month(x))

        # ensure we're working in the correct date range
        filtered_chunk = chunk[chunk["year"] > 2012]
        filtered_chunk = filtered_chunk[filtered_chunk["year"] < 2015]

        temp_df = pd.concat([temp_df, filtered_chunk])
        break
    
    missing_years = pd.concat([missing_years, temp_df], axis=0, ignore_index=True)



In [11]:
# feel like there should be an assert here but I'm not sure what it would be
df_prescriptions = pd.concat([df, missing_years], axis=0, ignore_index=True)

While doing analysis, we learned that a handful of county values in MA were missing. However, when we looked up the associated zip codes (02401, 02174), we learned that we could fill these values with Plymouth and Middlesex Counties, respectively.

In [12]:
df_prescriptions["date_test"] = df_prescriptions["TRANSACTION_DATE"].apply(lambda x: datetime.strptime(str(x), "%m%d%Y"))

In [13]:
df_prescriptions["year_test"] = df_prescriptions["TRANSACTION_DATE"].apply(lambda x: find_year(x))

In [14]:
df_prescriptions["TRANSACTION_DATE"]

0          1122009
1          8182006
2         11292006
3          2062007
4          3012007
            ...   
924654     2092013
924655     3132013
924656     3022013
924657     4032013
924658     2182013
Name: TRANSACTION_DATE, Length: 924659, dtype: int64

In [15]:
df_prescriptions[df_prescriptions["BUYER_STATE"] == "WA"].year.unique()

array([2006, 2007, 2012, 2008, 2010, 2011, 2009, 2014, 2013], dtype=int64)

In [16]:
# quick look at the null values (we checked, and these are all the values for which county is null)
df_prescriptions[df_prescriptions["BUYER_COUNTY"].isnull()].head()

,REPORTER_DEA_NO,BUYER_STATE,BUYER_ZIP,BUYER_COUNTY,TRANSACTION_CODE,DRUG_CODE,DRUG_NAME,QUANTITY,TRANSACTION_DATE,Product_Name,year,month,date_test,year_test


In [17]:
# replace values accordingly
df_prescriptions.loc[df_prescriptions["BUYER_ZIP"] == 2401, "BUYER_COUNTY"] = "PLYMOUTH"
df_prescriptions.loc[df_prescriptions["BUYER_ZIP"] == 2174, "BUYER_COUNTY"] = "MIDDLESEX"

In [18]:
# check null values again
df_prescriptions[df_prescriptions["BUYER_COUNTY"].isnull()]

assert len(df_prescriptions[df_prescriptions["BUYER_COUNTY"].isnull()]) == 0, "still have missing values for counties - double check code above"

## Clean up cause of death data

In [19]:
path = r'00_source_data/cause_of_death' # point to correct folder
filenames = glob.glob(path + "/*.txt") # select all text files in folder

df = pd.DataFrame() # empty df - will store data from all txt files

for f in filenames:
    temp = pd.read_csv(f, index_col=None, header=0, sep='\t')
    # we're getting some extraneous notes at the bottom - let's just drop based on county as these will only be null for these useless notes columns
    temp.dropna(subset={'County'}, inplace=True)
    
    df = pd.concat([df, temp], axis=0, ignore_index=True)

In [20]:
# helper functions to separate county and state

def abtract_state(county):
    """
    Args:
        county (str): county name

    Returns:
        str: state
    """
    return county.split(", ")[1]



def abstract_county(county):
    """
    Args:
        county (str): county name

    Returns:
        str: county
    """
    return county.split(", ")[0]

# apply functions to our df
df["State"] = df.apply(lambda x: abtract_state(x["County"]), axis=1)
df["County"] = df.apply(lambda x: abstract_county(x["County"]), axis=1)

# do not need notes column, let's just drop it here
df.drop(columns={"Notes"}, inplace=True)

df_cause_of_death = df.copy() # keep a copy of this df for later filtering

In [21]:
# now, let's filter our dataframe to be only the states we want
df_cause_of_death = df_cause_of_death[df_cause_of_death["State"].isin(states)]


## Adding in County Population data

[Census county pop. data, 2000-2010](https://www.census.gov/data/tables/time-series/demo/popest/intercensal-2000-2010-counties.html)<br>
[Census county pop. data, 2010-2019](https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html)<br>
For both, just select the appropriate states on the webpage. We will clean and merge as needed in this notebook.



#### Guide to cleaning - 2000s data

The way the 2000s excel files are formatted, we can clean the data in the following way

- load in with header=3
- drop null on any of the populations
    - notes at the bottom will be removed
- drop unnamed 1, 12, and 13
    - these contain redundant data about populations from specific dates
    - Unnamed 12 is 2010s pop - will be redundant as our next dataset has this as well. Using the newer data
- drop first row
    - state as a whole
- rename Unnamed: 0 to county


In [22]:
# init emmpty df for our population data
pops00 = pd.DataFrame()

# end goal - add every excel file in 00_source_data/county_pop/2000s to pops00

path = r"00_source_data/county_pop/2000s/" # point to correct folder
filenames = glob.glob(path + "*.xls")

for f in filenames:

    # read in current file with header = 3
    temp = pd.read_excel(f, header = 3)

    # regex to pull out state from filename
    r = re.search("(2000s)(.)(\w+)", f)[3]
    temp["State"] = r[:2].upper()
    
    # drop null on any of the years
    temp.dropna(subset=[2000], inplace=True)

    #drop useless columns
    temp.drop(columns={"Unnamed: 1", "Unnamed: 12", "Unnamed: 13"}, inplace=True)

    # drop first row
    temp = temp.iloc[1:, :]

    # rename some cols
    temp.rename(columns={"Unnamed: 0": "County"}, inplace=True)

    # remove period at beginning of each county
    temp["County"] = temp["County"].apply(lambda x: x[1:])

    pops00 = pd.concat([pops00, temp], axis=0, ignore_index=True)

# quick peek at the data
pops00.head()


,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,State
0,Adams County,350888.0,359816.0,370753.0,377464.0,384809.0,395146.0,406575.0,415746.0,424913.0,435700.0,CO
1,Alamosa County,14954.0,14956.0,15114.0,15067.0,15217.0,15236.0,15196.0,15180.0,15300.0,15289.0,CO
2,Arapahoe County,491482.0,502393.0,508936.0,513690.0,518971.0,524466.0,531619.0,542039.0,552461.0,563161.0,CO
3,Archuleta County,10020.0,10454.0,10885.0,11089.0,11266.0,11496.0,11937.0,12262.0,12250.0,12169.0,CO
4,Baca County,4501.0,4471.0,4336.0,4117.0,4064.0,3997.0,3933.0,3866.0,3806.0,3767.0,CO


#### Guide to cleaning - 2010s data

The way the 2010s excel files are formatted, we can clean the data in the following way

- load in with header=3
- drop null on any of the populations
    - notes at the bottom will be removed
- drop census, estimates base
- drop first row
    - state as a whole
- rename Unnamed: 0 to county


In [23]:
pops10 = pd.DataFrame()

# add every excel file in 00_source_data/county_pop/2000s to pops00

path = r"00_source_data/county_pop/2010s" # point to correct folder
filenames = glob.glob(path + "/*.xlsx")

for f in filenames:

    # read in current file with header = 3
    temp = pd.read_excel(f, header = 3)

    # regex to pull out state from filename
    r = re.search("(2010s)(.)(\w+)", f)[3]
    temp["State"] = r[:2].upper()
    
    # drop null on any of the years
    temp.dropna(subset=[2010], inplace=True)

    #drop useless columns
    temp.drop(columns={"Census", "Estimates Base"}, inplace=True)

    # drop first row
    temp = temp.iloc[1:, :]

    # rename some cols
    temp.rename(columns={"Unnamed: 0": "County"}, inplace=True)

    # remove period at beginning of each county
    temp["County"] = temp["County"].apply(lambda x: x[1:])

    # strip state from county
    temp["County"] = temp["County"].apply(lambda x: x.split(", ")[0])

    pops10 = pd.concat([pops10, temp], axis=0, ignore_index=True)

# quick peek at the data
pops10.head()


,County,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,State
0,Adams County,443691.0,452201.0,460558.0,469978.0,479946.0,490443.0,497734.0,503590.0,511354.0,517421.0,CO
1,Alamosa County,15515.0,15709.0,15680.0,15787.0,15803.0,15894.0,16053.0,16108.0,16248.0,16233.0,CO
2,Arapahoe County,574747.0,585968.0,596500.0,608467.0,619034.0,630984.0,638950.0,644478.0,651797.0,656590.0,CO
3,Archuleta County,12046.0,12021.0,12132.0,12216.0,12231.0,12387.0,12825.0,13295.0,13730.0,14029.0,CO
4,Baca County,3807.0,3778.0,3722.0,3656.0,3587.0,3555.0,3530.0,3554.0,3584.0,3581.0,CO


In [24]:
# melt both dfs to get tidy format
pops00 = pops00.melt(["County", "State"])
pops10 = pops10.melt(["County", "State"])

# rename columns accordingly
pops00.rename(columns={"variable": "Year", "value": "Population"}, inplace=True)
pops10.rename(columns={"variable": "Year", "value": "Population"}, inplace=True)

# concatenate the two dfs to get all our population data in one place
pops = pd.concat([pops00, pops10], ignore_index=True)


In [25]:
# check that we have the same number of counties between datasets
assert len(pops00["County"].unique()) == len(pops10["County"].unique())

# check that we have the same number of counties every year
# first, create a df with the number of counties per year
pops_county_check = pops.groupby(["State", "Year"])["County"].count().reset_index()

In [26]:
# group the sum of counties by year and state - will help us check if number of counties changes over the years
grouped_states = pops_county_check.groupby(["Year", "State"])["County"].sum().reset_index().rename(columns={"County": "county_count"})

# here's what this looks like
# we get a dataframe of states and years, with the number of counties in each state in each year
grouped_states.head()

,Year,State,county_count
0,2000,CO,64
1,2000,FL,67
2,2000,IL,102
3,2000,MA,14
4,2000,MD,24


In [27]:
# using the above query, we should be able to assert that the number of counties per year is the same
# below statement should always equal zero

assert (grouped_states.duplicated(subset=["Year", "State"]).sum() == 0)
#assert (grouped_states10.duplicated(subset=["Year", "State"]).sum() == 0)


# ensure no duplicate values
assert pops.duplicated().sum() == 0

# loop to check that every state has the same number of counties every year
for state in states:
    assert (pops[pops["State"] == state].Year.value_counts().nunique() == 1), f"error on {state}"

## trying to integrate fip numbers for a better merge

In [28]:
# load in fips data from external source
fips = pd.read_csv("https://github.com/ChuckConnell/articles/raw/master/fips2county.tsv", sep="\t")

In [29]:
# function to get key from value in our abbreviation dictionary
# will help us have consistent formatting across dataframes for merging purposes
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]


keys = get_keys_from_value(abbrev_to_us_state, 'Alabama')
keys # quick peek to make sure it worked

['AL']

In [30]:
# apply the above to entire fips dataframe
fips["state_abbrev"] = fips["StateName"].apply(lambda x: get_keys_from_value(abbrev_to_us_state, x)[0])

# filter fips to appropriate states, now that it's in the correct format
fips = fips[fips["state_abbrev"].isin(states)]

#### Further cleaning of values before merge

In [31]:
# helper function to get rid of the word county in pop df
def remove_county(x):

    if "County" in x:
        return x[:-7]
    else:
        return x


pops["county_test"] = pops["County"].apply(lambda x: remove_county(x))


# fix dona ana and la salle parish
pops["county_test"] = pops["county_test"].apply(lambda x: x.replace("Doña Ana", "Dona Ana"))
fips["CountyName"] = fips["CountyName"].apply(lambda x: x.replace("DoÃ±a Ana", "Dona Ana"))


#pops["county_test"] = pops["county_test"].apply(lambda x: x.replace("La Salle Parish", "La Salle"))


# rename county_test where state is texas and county is la salle to La Salle (TX)
pops.loc[(pops["State"] == "TX") & (pops["county_test"] == "La Salle"), "county_test"] = "La Salle County"
        

In [32]:
# change La Salle county name in fips to La Salle County
fips.loc[fips["CountyName"] == "La Salle", "CountyName"] = "La Salle County"
fips.loc[fips["CountyName"] == "LaSalle Parish", "CountyName"] = "La Salle Parish"
pops.loc[pops["county_test"] == "LaSalle Parish", "county_test"] = "La Salle Parish"



In [33]:
# final merge for population dataset & fip number dataset
pops_copy = pops.merge(fips[["state_abbrev", "CountyFIPS", "StateFIPS", "CountyName"]], left_on=["county_test", "State"], right_on=["CountyName", "state_abbrev"], how="outer", indicator=True)

In [34]:
# should never end up with anything left out of merge
assert len(pops_copy[pops_copy["_merge"] != "both"]) == 0

In [35]:
# add fip numbers to df_prescriptions

# create copies of both dfs so we have a checkpoint to access our old dfs
prescriptions_copy = df_prescriptions.copy()
fips_copy = fips.copy()

In [36]:
# make buyer_county all lowercase
prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.lower())

# do the same for fips
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: x.lower())

In [37]:
# remove county and parish from fips_copy

def remove_parish(x):

    if "parish" in x:
        return x[:-7]
    else:
        return x


# prescription dataset has similar format - match fips to this format
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: remove_county(x))
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: remove_parish(x))

def expand_saint(x):

    if "st." in x:
        return x.replace("st.", "saint")
    else:
        return x

# fix various other inconsistencies
# left only values first
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: expand_saint(x))

fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: x.replace("desoto", "de soto"))
prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.replace("desoto", "de soto"))
prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.replace("st john the baptist", "saint john the baptist"))

fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: x.replace("dekalb", "de kalb"))
prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.replace("dekalb", "de kalb"))

# fix right only values

prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.replace("desoto", "de soto"))



# function to remove apostrophes from county names
def remove_apostrophe(x):
    
    if "'" in x:
        return x.replace("'", "")
    else:
        return x


# apply to fips
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: remove_apostrophe(x))

# replace lasalle with la salle in fips copy
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: x.replace("lasalle", "la salle"))

# replace dewitt with de witt in prescriptions copy
prescriptions_copy["BUYER_COUNTY"] = prescriptions_copy["BUYER_COUNTY"].apply(lambda x: x.replace("dewitt", "de witt"))


In [38]:
prescriptions_fips = prescriptions_copy.merge(fips_copy, left_on=["BUYER_COUNTY", "BUYER_STATE"], right_on=["CountyName", "state_abbrev"], how="outer", indicator=True)

# capitalize year and month columns
prescriptions_fips.rename(columns={"year": "Year", "month": "Month"}, inplace=True)

In [39]:
# STILL HAVE ALL WASHINGTON YEARS HERE
prescriptions_fips[prescriptions_fips["BUYER_STATE"] == "WA"].Year.value_counts()

2013.0    56182
2014.0    54377
2011.0     1543
2012.0     1261
2010.0     1012
2008.0      912
2009.0      911
2007.0      895
2006.0      791
Name: Year, dtype: int64

#### Imputing missing values

Since we have plenty of values joined with right_only indicator status, we know that some counties in our FIPS dataset is not merging correctly to our prescriptions dataset. Let's take one example of missing data - San Juan County in Washington. When exploring the Washington Post's website on prescription data and selecting for this county individually, we can see that the data does, in fact, exist here. However, we see an exceptionally low rate of pills prescribed (32 pills per person per year, in this case). Upon looking at some other examples, we can see that the counties joining with right_only below are likely missing from the Washington Post data due to having such small numbers.

Since any given missing county does not have data available, we need to find a way to impute these values.

In [40]:
prescriptions_fips.Year.isnull().sum()

378

In [41]:
# let's look at an example of our missing data
wa = df_prescriptions[df_prescriptions["BUYER_STATE"] == 'WA']

wa["BUYER_COUNTY"].value_counts()

KING            27359
SNOHOMISH       13079
PIERCE          11853
SPOKANE         11069
CLARK            8521
YAKIMA           5826
BENTON           4536
THURSTON         3825
KITSAP           3553
COWLITZ          3049
WHATCOM          2868
SKAGIT           2536
CLALLAM          1723
WALLA WALLA      1701
CHELAN           1502
GRAYS HARBOR     1355
GRANT            1333
FRANKLIN         1247
LEWIS            1242
STEVENS          1060
MASON            1054
ISLAND            964
ASOTIN            931
KITTITAS          839
WHITMAN           827
OKANOGAN          742
DOUGLAS           553
PEND OREILLE      464
PACIFIC           379
JEFFERSON         353
SAN JUAN          291
LINCOLN           274
ADAMS             267
COLUMBIA          234
KLICKITAT         157
SKAMANIA          126
FERRY             104
WAHKIAKUM          46
GARFIELD           42
Name: BUYER_COUNTY, dtype: int64

In [42]:
assert prescriptions_fips[prescriptions_fips["_merge"] == "left_only"].shape[0] == 0

#### Not all counties joining back to prescription dataset

This could be okay, but I want to do a quick check that there are just not records for these counties. To do this, I'll take a small sample of counties in our FIPS dataset that did NOT merge properly to the prescriptions dataset, and search each one manually in the prescription dataset. I will search various different ways the counties could be transcribed, as well as google the county to ensure there are no secondary names for the same county.

In [43]:
prescriptions_fips[prescriptions_fips["_merge"] != "both"].columns

Index(['REPORTER_DEA_NO', 'BUYER_STATE', 'BUYER_ZIP', 'BUYER_COUNTY',
       'TRANSACTION_CODE', 'DRUG_CODE', 'DRUG_NAME', 'QUANTITY',
       'TRANSACTION_DATE', 'Product_Name', 'Year', 'Month', 'date_test',
       'year_test', 'StateFIPS', 'CountyFIPS_3', 'CountyName', 'StateName',
       'CountyFIPS', 'StateAbbr', 'STATE_COUNTY', 'state_abbrev', '_merge'],
      dtype='object')

Need a new solution to the above - should impute the values somehow

In [44]:
#prescriptions_fips = prescriptions_fips[prescriptions_fips["_merge"] != "right_only"]

# may have to add explanation for this higher up
# re add assertion once value imputer is done
#assert len(prescriptions_copy) == len(prescriptions_fips)


### adding fips to our cause of death data

In [45]:
# create copies of both dfs

cause_of_death_copy = df_cause_of_death.copy()
fips_copy = fips.copy()

In [46]:
# remove county once again
cause_of_death_copy["County"] = cause_of_death_copy["County"].apply(lambda x: remove_county(x))


# clean some other miscellaneous values up

cause_of_death_copy["County"] = cause_of_death_copy["County"].apply(lambda x: x.replace("LaSalle Parish", "La Salle Parish"))
cause_of_death_copy["County"] = cause_of_death_copy["County"].apply(lambda x: x.replace("DeBaca", "De Baca"))
cause_of_death_copy["County"] = cause_of_death_copy["County"].apply(lambda x: x.replace("La Salle", "La Salle County"))
cause_of_death_copy["County"] = cause_of_death_copy["County"].apply(lambda x: x.replace("La Salle County Parish", "La Salle Parish"))



# expand mckean to mc kean in fips_copy
fips_copy["CountyName"] = fips_copy["CountyName"].apply(lambda x: x.replace("McKean", "Mc Kean"))


In [47]:
cause_of_death_fips = cause_of_death_copy.merge(fips_copy, left_on=["County", "State"], right_on=["CountyName", "state_abbrev"], how="outer", indicator=True)

#### Not all counties joining back to cause of death dataset

If the number of people in a given category (eg. one county/year/cause of death category) is less than 10, those records do not appear in this data. There is also a technicality in the number of total deaths vs. drug deaths (which we are interested in).

The example we are given is that if a county has 20 deaths unrelated to drugs and alcohol, and only 7 related to alcohol, only the former figure will be reported. In the next notebook (pick_states.ipynb), we will filter by cause of death. In this notebook, since we still have all causes of death, we will impute for every missing value.

To impute this data, we will fill in missing values with **a random integer from 0 to 9**. We thought of drawing from a normal distribution, but this implies negative values could be attained. We could take their absolute values to negate this effect, but then we are no longer drawing from a *true* normal distribution, so we chose to pick random values in our range.

In [48]:
# function to replace null value with a random integer from 0 to 10 with a normal distribution
def value_imputer(x):
    if pd.isnull(x):
        return random.randint(0, 9)
    else:
        return x

cause_of_death_fips["Deaths"] = cause_of_death_fips["Deaths"].apply(lambda x: value_imputer(x))

In [49]:
# quick look at our new imputed data
cause_of_death_fips[cause_of_death_fips["_merge"] != "both"].Deaths.value_counts()

6    1
4    1
7    1
1    1
3    1
9    1
Name: Deaths, dtype: int64

### Adding Population to final DataFrames

For pop_fips, cause_of_death_fips, and prescription_fips. Steps needed:

- Create unique ID from county FIPS and state FIPS
- Merge population dataset based on this

In [50]:
cause_of_death_fips = cause_of_death_fips[cause_of_death_fips["_merge"] == "both"]
#pops_copy = pops_copy[cause_of_death_fips["_merge"] == "both"]
prescriptions_fips = prescriptions_fips[prescriptions_fips["_merge"] == "both"]


# drop merge columns
cause_of_death_fips.drop(columns=["_merge"], inplace=True)
prescriptions_fips.drop(columns=["_merge"], inplace=True)
pops_copy.drop(columns=["_merge",], inplace=True)

In [51]:
# create unique FIP from county and state fips

cause_of_death_fips["FIP_unique"] = cause_of_death_fips["CountyFIPS"].apply(lambda x: str(x)) + cause_of_death_fips["StateFIPS"].apply(lambda x: str(x))
prescriptions_fips["FIP_unique"] = prescriptions_fips["CountyFIPS"].apply(lambda x: str(x)) + prescriptions_fips["StateFIPS"].apply(lambda x: str(x))
pops_copy["FIP_unique"] = pops_copy["CountyFIPS"].apply(lambda x: str(x)) + pops_copy["StateFIPS"].apply(lambda x: str(x))


In [52]:
# TODO: add some sort of assert here. not sure what it should be yet


In [53]:
# create final prescriptions dataset with populations
# can safely left join here, because we only need records in the prescriptions dataset
prescriptions = prescriptions_fips.merge(pops_copy, on=["FIP_unique", "Year"], how="left", indicator=True)

assert (prescriptions["_merge"] == "both").all()

In [54]:
# STILL HAVE WASHINGTON YEARS HERE
prescriptions[prescriptions["BUYER_STATE"] == "WA"].Year.value_counts()

2013.0    56182
2014.0    54377
2011.0     1543
2012.0     1261
2010.0     1012
2008.0      912
2009.0      911
2007.0      895
2006.0      791
Name: Year, dtype: int64

In [55]:
# one more assert to check length
assert len(prescriptions) == len(prescriptions_fips)


In [56]:
# drop some useless columns
prescriptions.drop(columns=["_merge", "CountyName_y", "StateFIPS_y", "CountyFIPS_y","state_abbrev_y", "County", "CountyFIPS_3"], inplace=True)

# rename x columns
prescriptions.rename(columns={"CountyName_x": "CountyName", "StateFIPS_x": "StateFIPS", "CountyFIPS_x": "CountyFIPS", "state_abbrev_x": "state_abbrev"}, inplace=True)


In [57]:
# create final cause of death dataset with populations
# can safely left join here, because we only need records in the cause of death dataset
cause_of_death = cause_of_death_fips.merge(pops_copy, on=["FIP_unique", "Year"], how="left", indicator=True)

assert cause_of_death_fips.Deaths.isnull().sum() == 0

In [58]:
# drop some useless columns
cause_of_death.drop(columns=["_merge", "CountyName_y", "StateFIPS_y", "CountyFIPS_y","state_abbrev_y", "County_y", "CountyFIPS_3", "State_y"], inplace=True)

# rename x columns
cause_of_death.rename(columns={"County_x": "County", "Year_x": "Year", "State_x": "State", "StateFIPS_x": "StateFIPS", "CountyFIPS_x": "CountyFIPS", "state_abbrev_x": "state_abbrev", "CountyName_x": "CountyName"}, inplace=True)


In [59]:
# asserts to make sure we didn't lose any records from our original datasets

assert len(df_cause_of_death) == len(cause_of_death)
assert len(df_prescriptions) == len(prescriptions)

#### Export main, unjoined datasets in case we need them

In [60]:
cause_of_death.to_csv("20_intermediate_files/cause_of_death_clean.csv", index=False)
prescriptions.to_csv("20_intermediate_files/arcos_all_washpost_clean.csv", index=False)

## Final 3 datasets

We should have: (UNSURE IF WE SHOULD EXTEND DATE RANGES, CURRENTLY 3 YEARS BEFORE AND AFTER POLICY IMPLEMENTATION)

- Florida and Georgia 2007 - 2013
- Texas and Oklahoma 2004 - 2010
- Washington and Oregon 2009 - 2015

### Drug overdose - broken down by state

In [ ]:
# Florida and Georgia

prescriptions_fl = prescriptions.copy()
prescriptions_wa = prescriptions.copy()

prescriptions_fl = prescriptions_fl[(prescriptions_fl["BUYER_STATE"] == "FL") | (prescriptions_fl["BUYER_STATE"].isin(fl_states))]
prescriptions_wa = prescriptions_wa[(prescriptions_wa["BUYER_STATE"] == "WA") | (prescriptions_wa["BUYER_STATE"]).isin(wa_states)]



# filter appropriate years
fl_start = 2007
fl_end = 2013

# tx will only be used for overdose deaths
tx_start = 2004
tx_end = 2010

wa_start = 2009
wa_end = 2015


prescriptions_fl = prescriptions_fl[(prescriptions_fl["Year"] >= fl_start) & (prescriptions_fl["Year"] <= fl_end)]
prescriptions_wa = prescriptions_wa[(prescriptions_wa["Year"] >= wa_start) & (prescriptions_wa["Year"] <= wa_end)]


In [ ]:
prescriptions_fl

NameError: name 'prescriptions_fl' is not defined

### Cause of death - broken down by state

In [ ]:
deaths_fl = cause_of_death.copy()
deaths_tx = cause_of_death.copy()
deaths_wa = cause_of_death.copy()

deaths_fl = deaths_fl[(deaths_fl["StateName"] == "Florida") | (deaths_fl["State"].isin(fl_states))]
deaths_tx = deaths_tx[(deaths_tx["StateName"] == "Texas") | (deaths_tx["State"].isin(tx_states))]
deaths_wa = deaths_wa[(deaths_wa["StateName"] == "Washington") | (deaths_wa["State"].isin(wa_states))]

deaths_fl = deaths_fl[(deaths_fl["Year"] >= fl_start) & (deaths_fl["Year"] <= fl_end)]
deaths_tx = deaths_tx[(deaths_tx["Year"] >= tx_start) & (deaths_tx["Year"] <= tx_end)]  
deaths_wa = deaths_wa[(deaths_wa["Year"] >= wa_start) & (deaths_wa["Year"] <= wa_end)]

In [ ]:
# final assert to check years

assert prescriptions_fl.Year.unique().tolist() == list(range(fl_start, fl_end + 1))
assert prescriptions_wa.Year.unique().tolist() == list(range(wa_start, wa_end + 1))

assert deaths_fl.Year.unique().tolist() == list(range(fl_start, fl_end + 1))
assert deaths_tx.Year.unique().tolist() == list(range(tx_start, tx_end + 1))
assert deaths_wa.Year.unique().tolist() == list(range(wa_start, wa_end + 1))

### export all to csv

In [ ]:
prescriptions_fl.to_csv("20_intermediate_files/prescriptions_fl.csv", index=False)
prescriptions_wa.to_csv("20_intermediate_files/prescriptions_wa.csv", index=False)

deaths_fl.to_csv("20_intermediate_files/deaths_fl.csv", index=False)
deaths_tx.to_csv("20_intermediate_files/deaths_tx.csv", index=False)
deaths_wa.to_csv("20_intermediate_files/deaths_wa.csv", index=False)

## Notes for the group

- may need to filter out a couple more columns - haven't done this yet as I don't want to accidentally delete something we need
- overdose data is only broken down by year unless i messed something up - overdose analysis will have to be less granular